In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

/kaggle/input/binary-buzz-cyber-labs-ml-25/train.csv
/kaggle/input/binary-buzz-cyber-labs-ml-25/test.csv


In [2]:
train_set = pd.read_csv("/kaggle/input/binary-buzz-cyber-labs-ml-25/train.csv").to_numpy()
test_set2 = pd.read_csv("/kaggle/input/binary-buzz-cyber-labs-ml-25/test.csv").to_numpy()

y_label = train_set[ : ,-1]
train_set = train_set[ : , :-1] 

print("The shape of train_set is :", train_set.shape)
print("The shape of test_set2 is :", test_set2.shape)
print("The shape of y_label is", y_label.shape)

The shape of train_set is : (2000, 3073)
The shape of test_set2 is : (2000, 3073)
The shape of y_label is (2000,)


In [3]:
X_train,X,y_train,y = train_test_split(train_set, y_label, test_size = 0.4)
X_test,X_cv,y_test,y_cv = train_test_split(X, y, test_size = 0.5)

print(f"The shape of X_train = {X_train.shape}, X_test = {X_test.shape}, X_cv = {X_cv.shape}")
print(f"The shape of y_train = {y_train.shape}, y_test = {y_test.shape}, y_cv = {y_cv.shape}")

The shape of X_train = (1200, 3073), X_test = (400, 3073), X_cv = (400, 3073)
The shape of y_train = (1200,), y_test = (400,), y_cv = (400,)


In [4]:
def balanced_accuracy(y, yhat):
    TPR = np.sum( (y == 1) & (yhat == 1) ) / np.sum( y == 1 )
    TNR = np.sum( (y == 0) & (yhat == 0) ) / np.sum( y == 0 )
    
    accuracy = (TPR + TNR) / 2
    return accuracy

In [5]:
np.random.seed(42)

C = [0.001, 0.01, 0.1, 1, 8, 10, 100]

accuracy_val = np.zeros_like( C )
accuracy_val_test = np.zeros_like( C )
for i,C_val in enumerate(C):
    model = LogisticRegression(
        C = C_val,
        penalty = 'l2',
        solver = 'liblinear',
        random_state = 42
    )
    
    model.fit(X_train, y_train)
    y_cv_pred = model.predict(X_cv)
    accuracy = balanced_accuracy(y_cv, y_cv_pred)
    accuracy_val[i] = accuracy

    y_test_pred = model.predict(X_test)
    accuracy = balanced_accuracy(y_test, y_test_pred)
    accuracy_val_test[i] = accuracy

i = np.argmax( accuracy_val )
C_val = C[ np.argmax(accuracy_val) ]
lambda_ = 1 / C_val
print("The maximum accuracy in the Cross Validation Set : ", accuracy_val[i] )
print( f"Accuracy in the Test Set for given regularisation parametre {lambda_} is { accuracy_val_test[i] }" )

model = LogisticRegression(
    C = C_val,
    penalty = 'l2',
    solver = 'liblinear',
    random_state = 42
)

model.fit(X_train, y_train)
y_pred = model.predict(test_set2)
final_predicted_list = np.c_[test_set2, y_pred] 
print( " the first 5 final predicted examples of test set is : \n", final_predicted_list[0:5] )

test_df = pd.read_csv("/kaggle/input/binary-buzz-cyber-labs-ml-25/test.csv")

test_ids = test_df['id'].values

submission = pd.DataFrame({
    'id': test_ids, 
    'target': y_pred.ravel()
})

submission.to_csv('submission.csv', index=False)

print("Success! 'submission.csv' has been created.")

The maximum accuracy in the Cross Validation Set :  0.79763483642794
Accuracy in the Test Set for given regularisation parametre 100.0 is 0.7027991031999457
 the first 5 final predicted examples of test set is : 
 [[3731  134  110 ...  105   52    0]
 [1052   25   24 ...   13   15    0]
 [3050   84   90 ...  163  151    0]
 [2184   24   17 ...   29   26    0]
 [2957  136  149 ...   55   24    0]]
Success! 'submission.csv' has been created.
